In [6]:
from tdmclient import ClientAsync, aw
import time

import numpy as np
import math

from obstacle_avoidance import *
from PIL import Image
from utils.data import *



In [ ]:
client = ClientAsync()
node = await client.wait_for_node()
aw(node.lock())


In [5]:
#import map from image and set start/goal
im = Image.open('b.png', 'r')
im = im.convert('L')
grid = np.array(im)
grid = (grid !=255)*1
max_x, max_y = grid.shape[0], grid.shape[1] # Size of the map

goal = Point(1,7)
start = Point(19,7)
base_dir = 0

thym = Robot(start, base_dir)



In [120]:
#movement simple décomposé en rotation et deplacement

def turn_angle(angle):
    "turn thymio clockwise to a given angle given in degrees, convert to -180 to 180 degrees, speed is -200;200"
    TURN = 4530 #time in ms to do 360 at -200,200
    dir = 1
    if angle < 0:
        dir = -1
        angle = abs(angle)
    if angle == 0:
        return
    timing = TURN//(360/angle)/2
    
    v = {
    "motor.left.target": [dir*400],
    "motor.right.target": [-dir*400],
    }
    aw(node.set_variables(v) )
    aw(client.sleep(timing/1000)) 
    v = {
    "motor.left.target": [0],
    "motor.right.target": [0],
    }
    aw(node.set_variables(v))

def move_forward(dist):
    "move forward a given distance in centimeters"
    METER = 14700 #time in ms to move 1m at 200,200
    dir = 1
    if dist < 0:
        dir = -1
        dist = abs(dist)

    timing = METER//(100/dist)/2
    
    
    v = {
    "motor.left.target": [dir*400],
    "motor.right.target": [dir*400],
    }
    aw(node.set_variables(v) )
    aw(client.sleep(timing/1000)) 
    v = {
    "motor.left.target": [0],
    "motor.right.target": [0],
    }
    aw(node.set_variables(v))

def move_step(next_x, next_y, current_dir):
    "move with given direction in 8 possibilities. Turn first then move"
    dist = 1 #set to 10 cm but depends on size of grid
    if next_x != 0 and next_y != 0:
        dist *=  2**(1/2)           #if diagonal
    
    if next_x == 0 and next_y == 1:
        angle = 0 - current_dir
    elif next_x == 0 and next_y == -1:
        angle = 180 - current_dir
    elif next_x == 1 and next_y == 1:
        angle = 45 - current_dir
    elif next_x == 1 and next_y == 0:
        angle = 90 - current_dir                
    elif next_x == 1 and next_y == -1:
        angle = 135 - current_dir
    elif next_x == -1 and next_y == 1:
        angle = -45 - current_dir
    elif next_x == -1 and next_y == 0:
        angle = -90 - current_dir   
    elif next_x == -1 and next_y == -1:
        angle = -135 - current_dir     
    
    angle %= 360
    if angle > 180:
        angle -= 360
    
    turn_angle(angle)
    move_forward(dist)
    return angle + current_dir


#test avec path de map.py
path = np.array([[19,18,17,16,15,14,13,12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1] ,[ 7,  7,  7,  7,  7,  7,  7,  6,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  7]])
    
curr = 0

for i in range(path.shape[1]-1):
    curr = move_step(path[0][i+1]-path[0][i], path[1][i+1]-path[1][i],curr)
    



In [181]:
def test_control(pos, goal, orientation):
    GAIN = 2
    SPEED = 200
    METER = 14700

    dist = ((goal[0]-pos[0])**2+(goal[1]-pos[1])**2)**(1/2)
    print(dist)
    if (goal[0]-pos[0]) == 0:
        if goal[1]-pos[1] > 0:
            angle = 90
        else:
            angle = -90
    else:
        angle = 180*((goal[1]-pos[1]) < 0) + math.atan((goal[1]-pos[1])/(goal[0]-pos[0]))/np.pi*180 - orientation
        print(angle)
    speedR = int(SPEED - GAIN * angle)
    speedL = int(SPEED + GAIN * angle)
    v = {
    "motor.left.target": [speedL],
    "motor.right.target": [speedR],
    }
    aw(node.set_variables(v))
    aw(client.sleep(METER//(10/dist)/1000)) 
    v = {
    "motor.left.target": [0],
    "motor.right.target": [0],
    }
    aw(node.set_variables(v))
        
pos = (0,0)
goal = (0,1)
orient = 0



test_control(pos, goal, orient)


1.0


In [153]:
v = {
"motor.left.target": [0],
"motor.right.target": [0],
}
aw(node.set_variables(v))


In [2]:
await node.unlock()

{'error_code': 2}

AttributeError: 'tuple' object has no attribute 'left'